Necessary Libraries

In [12]:
import requests
import pandas as pd
from pathlib import Path

API_KEY = "36e399c8efddfc967109cac3eaba15c58ccf99d9"

Retrieve Data

In [13]:
years = range(2011, 2025)
all_data = []

for year in years:
    url = f"https://api.census.gov/data/{year}/acs/acs1"
    params = {
        "get": "NAME,B12501_001E,B12501_005E,B12501_010E,B12503_005E,B12503_010E",
        "for": "state:*",
        "key": API_KEY
    }

    response = requests.get(url, params=params)

    try:
        data = response.json()
    except ValueError:
        print(f"{year}: JSON failed — skipped")
        continue

    df = pd.DataFrame(data[1:], columns=data[0])
    df["year"] = year
    #print(df)
    all_data.append(df)


2020: JSON failed — skipped


Clean Data

In [21]:
#Declare empty dataframe
final_df = pd.DataFrame()

# Combine all years
final_df = pd.concat(all_data, ignore_index=True)

# Rename for clarity
final_df = final_df.rename(columns={
    "state": "state_num",
    "NAME": "state",
    "B12501_001E": "population_over_15",
    "B12501_005E": "married_males_last_year",
    "B12501_010E": "married_females_last_year",
    "B12503_005E": "divorced_males_last_year",
    "B12503_010E": "divorced_females_last_year"
})

cols = [
    "population_over_15",
    "married_males_last_year",
    "married_females_last_year",
    "divorced_males_last_year",
    "divorced_females_last_year"
]

for col in cols:
    final_df[col] = pd.to_numeric(final_df[col], errors="coerce")

# Compute totals
final_df["married_last_year"] = (
    final_df["married_males_last_year"] +
    final_df["married_females_last_year"]
)

final_df["divorced_last_year"] = (
    final_df["divorced_males_last_year"] +
    final_df["divorced_females_last_year"]
)

# Marriage rate per 1,000
final_df['marriage_rate_per_1000'] = (final_df['married_last_year'] / final_df['population_over_15']) * 1000

# Divorce rate per 1,000
final_df['divorce_rate_per_1000'] = (final_df['divorced_last_year'] / final_df['population_over_15']) * 1000

final_df = final_df[['state','year','married_last_year','marriage_rate_per_1000',
          'divorced_last_year','divorce_rate_per_1000','population_over_15']]

final_df.head()

,state,year,married_last_year,marriage_rate_per_1000,divorced_last_year,divorce_rate_per_1000,population_over_15
0,Alabama,2011,75517,19.489475,50452.0,13.020684,3874758
1,Alaska,2011,15537,27.491233,7469.0,13.215680,565162
2,Arizona,2011,94665,18.455343,55091.0,10.740224,5129409
3,Arkansas,2011,49544,21.151173,28500.0,12.167133,2342376
4,California,2011,489201,16.272858,239728.0,7.974349,30062390


Save to CSV

In [23]:
final_df.to_csv("../data/Marriage_Divorce_rates.csv")